In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

%matplotlib inline

In [ ]:
def visualize(data, figsize=(15, 15)):
    
    plt.figure(figsize=figsize)
    
    step_value = pd.DataFrame(data.groupby(by='episode')["step"].max())
    step_value.reset_index(inplace=True)

    plt.title("last step hist")
    sns.histplot(data=step_value, x=step_value.step, kde=True, bins=5)
    plt.show()

    # 상관관계 시각화 & des
    def heatmap(data):
        df = data.drop(columns=['reward', 'step'])
        corr = df.corr()
        fig, ax = plt.subplots(figsize=figsize)

        # 삼각형 마스크를 만든다(위 쪽 삼각형에 True, 아래 삼각형에 False)
        mask = np.zeros_like(corr, dtype=np.bool)
        mask[np.triu_indices_from(mask)] = True

        # 히트맵을 그린다
        sns.heatmap(
            corr,
            cmap='RdYlBu_r',
            annot=True,  # 실제 값을 표시한다
            mask=mask,  # 표시하지 않을 마스크 부분을 지정한다
            linewidths=.5,  # 경계면 실선으로 구분하기
            cbar_kws={"shrink": .5},  # 컬러바 크기 절반으로 줄이기
            vmin=-1,
            vmax=1  # 컬러바 범위 -1 ~ 1
        )

        plt.title('Correation Matrix', fontsize=20)
        plt.show()

    heatmap(data)

    # 그래프
    col_names = data.columns.to_list()
    col_names.remove('step')
    col_names.remove('episode')
    col_names.remove('reward')
    col_names.remove('done')
    col_names.remove('action')

    for col in col_names:
        fig, ax = plt.subplots(figsize=(15, 15))
        sns.histplot(x=data[col], kde=True)

    # 기초 통계량
    print(data.describe())

In [ ]:
data = pd.read_csv("DDQNv2.csv")
visualize(data)

In [ ]:
data.groupby(by='episode')["step"].describe()

In [ ]:
step_value = pd.DataFrame(data.groupby(by='episode')["step"].max())
step_value.reset_index(inplace=True)
step_value

In [ ]:
sns.histplot(data=step_value,x=step_value.step,kde=True,bins=5)

In [ ]:
plt.title("last step hist")
sns.histplot(x=data["step"],kde=True)